In [1]:
%load_ext autoreload
# Enable autoreload for all modules
%autoreload 2
from spot_control_suite.util.terrain_utils import get_terrain_urdf
from spot_control_suite.util.asset_utils import get_asset_path, AddSpotRemote

import numpy as np
from pydrake.all import(Meldis, 
                        RobotDiagramBuilder, 
                        AddDefaultVisualization, 
                        Rgba, 
                        Sphere, 
                        RigidTransform,
                        RotationMatrix)
from functools import partial

In [2]:

resolution = [20, 20]
height = np.random.rand(*resolution)*0.1
name = 'test_trimesh'
length = 1
width = 1

path_urdf, points, triangles = get_terrain_urdf(length, width, height, name)

radius = 0.2
ball_urdf = f'''
<robot name="ball_in_boxes">
  <link name="movable">
    <collision name="sphere">
      <geometry><sphere radius="{radius:.2f}"/></geometry>
    </collision>
    <visual name="sphere_vis">
      <geometry><sphere radius="{radius:.2f}"/></geometry>
    </visual>
  </link>
  <link name="for_joint"/>
  <link name="for_joint2"/>
  <joint name="x" type="prismatic">
    <axis xyz="1 0 0"/>
    <limit lower="-2" upper="2"/>
    <parent link="world"/>
    <child link="for_joint"/>
  </joint>
  <joint name="y" type="prismatic">
    <axis xyz="0 1 0"/>
    <limit lower="-2" upper="2"/>
    <parent link="for_joint"/>
    <child link="for_joint2"/>
  </joint>
  <joint name="z" type="prismatic">
    <axis xyz="0 0 1"/>
    <limit lower="-2" upper="2"/>
    <parent link="for_joint2"/>
    <child link="movable"/>
  </joint>
</robot>'''

In [4]:

#meshcat = StartMeshcat()
meldis = Meldis()
meshcat = meldis.meshcat
builder = RobotDiagramBuilder()
plant = builder.plant()
scene_graph = builder.scene_graph()
parser = builder.parser()
AddSpotRemote(parser)
model = parser.AddModels(path_urdf)
spot_urdf = get_asset_path() + '/spot/spot_with_arm_and_floating_base_actuators.urdf'
model2 = parser.AddModels(spot_urdf)
frame = plant.GetFrameByName(f"{name}", model[0])
plant.WeldFrames(plant.world_frame(), frame)
plant.Finalize()
visualizer = AddDefaultVisualization(builder.builder(), meshcat)
diagram = builder.Build()
diagram_context = diagram.CreateDefaultContext()
plant_context = plant.GetMyContextFromRoot(diagram_context)
diagram.ForcedPublish(diagram_context)

INFO:drake:Meldis is listening for LCM messages at udpm://239.255.76.67:7667?ttl=0
INFO:drake:Meshcat listening for connections at http://localhost:7001
INFO:drake:PackageMap: Downloading https://github.com/wrangel-bdai/spot_ros2/archive/20965ef7bba98598ee10878c7b54e6ef28a300c6.tar.gz


In [4]:
import ipywidgets as widgets
q = np.zeros(plant.num_positions()) 
sliders = []
for i in range(plant.num_positions()):
    q_low = plant.GetPositionLowerLimits()[i]*0.99
    q_high = plant.GetPositionUpperLimits()[i]*0.99
    sliders.append(widgets.FloatSlider(min=q_low, max=q_high, value=0, step=0.0001, description=f"q{i}"))
col_col =  Rgba(0.8, 0.0, 0, 0.5)    
col_free =  Rgba(0.0, 0.8, 0.5, 0.5) 
def showres(qvis):
    plant.SetPositions(plant_context, qvis)
    diagram.ForcedPublish(diagram_context)
    query = plant.get_geometry_query_input_port().Eval(plant_context)
    col = query.HasCollisions()
    if col:
        meshcat.SetObject(f"/drake/visualizer/shunk",
                                   Sphere(0.2),
                                   col_col)
    else:
        meshcat.SetObject(f"/drake/visualizer/shunk",
                                   Sphere(0.2),
                                   col_free)
    meshcat.SetTransform(f"/drake/visualizer/shunk",
                                   RigidTransform(RotationMatrix(),
                                                  np.array([0,0,0.5])))
    return col

def handle_slider_change(change, idx):
    q[idx] = change['new']
    showres(q)
idx = 0
for slider in sliders:
    slider.observe(partial(handle_slider_change, idx = idx), names='value')
    idx+=1

for slider in sliders:
    display(slider)

FloatSlider(value=0.0, description='q0', max=1.98, min=-1.98, step=0.0001)

FloatSlider(value=0.0, description='q1', max=1.98, min=-1.98, step=0.0001)

FloatSlider(value=0.0, description='q2', max=1.98, min=-1.98, step=0.0001)